In [1]:
using Flux, Statistics
using Flux: crossentropy, throttle, logitcrossentropy
using Base.Iterators: repeated, partition, flatten
using DelimitedFiles
using LinearAlgebra
using RDatasets
using GLM
using BSON: @save, @load
using Random
using Base.Threads 
using Plots

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/jupyter-rubin/.julia/packages/CUDA/mVgLI/src/initialization.jl:42


In [2]:
function getEdepGateBarA0(number, angle)
    folder = "/home/jupyter-rubin/dosimeter/gate_data/bars/"
    angle = lpad(angle,3,"0")
    fileName = filter(x->occursin(string("A", angle), x), readdir(folder))[number]
    filePath = string(folder, fileName)
    return Int.(round.(reshape(readdlm(filePath)[4:67,2], (8,8))))
end

getEdepGateBarA0 (generic function with 1 method)

In [3]:
getEdepGateBarA0(1, 0)

8×8 Matrix{Int64}:
 2  3   2  379  382   3  0  0
 0  2   2  304  404   2  2  0
 2  2   5  320  341   2  0  0
 2  1  23  399  375   3  4  0
 1  3   6  369  484   7  1  0
 0  2   2  374  469   4  1  2
 1  1   3  253  421   4  1  2
 1  0  29  156  193  20  3  1

In [4]:
function getEdepGateBarA90(number, angle)
    folder = "/home/jupyter-rubin/dosimeter/gate_data/bars/"
    angle90 = lpad(angle+90, 3, "0")
    fileName90 = filter(x->occursin(string("A", angle90), x), readdir(folder))[number]
    filePath90 = string(folder, fileName90)
    return return Int.(round.(reshape(readdlm(filePath90)[4:67,2], (8,8))))
end

getEdepGateBarA90 (generic function with 1 method)

In [5]:
function getEdepGateGrid(number, angle)
    folder = "/home/jupyter-rubin/dosimeter/gate_data/cubes/"
    angle = lpad(angle, 3, "0")
    fileName = filter(x->occursin(string("A", angle), x), readdir(folder))[number]
    filePath = string(folder, fileName)
    return Int.(round.(readdlm(filePath)[4:515,2]))
end

getEdepGateGrid (generic function with 1 method)

In [6]:
getEdepGateGrid(1, 45)

512-element Vector{Int64}:
 1
 0
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

dosimeter/gate_data/bars/W05__H05__A000_bars_extracted.txt

In [7]:
function getEdepOhioBarA0(number)
    folder = "/home/jupyter-rubin/dosimeter/ohio_data/extracted_data/"
    fileName = filter(x->occursin(r"R...txt", x), readdir(folder))[number]
    filePath = string(folder,fileName)
    return Int.(round.(reshape(readdlm(filePath)[4:67,2], (8,8))))
end

getEdepOhioBarA0 (generic function with 1 method)

In [8]:
#getEdepOhioBarA0(1)

In [9]:
function getEdepOhioBarA90(number)
    folder = "/home/jupyter-rubin/dosimeter/ohio_data/extracted_data/"
    fileName = filter(x->occursin(r"R.._r.txt", x), readdir(folder))[number]
    filePath = string(folder,fileName)
    return Int.(round.(reshape(readdlm(filePath)[4:67,2], (8,8))))
end

getEdepOhioBarA90 (generic function with 1 method)

In [10]:
#getEdepOhioBarA90(1)

In [11]:
function normalize(inputArray)
    min = minimum(inputArray)
    inputArray = inputArray.-min
    max = maximum(inputArray)/2048
    inputArray = inputArray./max
    return Int.(round.(inputArray))
end

normalize (generic function with 1 method)

In [12]:
#normalize(getEdepOhioBarA90(1))

In [13]:
#normalize(getEdepOhioBarA0(1))

In [14]:
function makeTrainingData()
    numRuns = 15
    folder = "/home/jupyter-rubin/dosimeter/gate_data/bars/"
    trainingSize = size(filter(x->occursin("A000", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8, 8, 2, trainingSize*numRuns)
    for i = 1:trainingSize
        for r = 1:15
            trainingArray[:, :, 1, (i-1)*numRuns + r] = getEdepGateBarA0(i, (r-1)*15)
            trainingArray[:, :, 2, (i-1)*numRuns + r] = getEdepGateBarA90(i, (r-1)*15)
        end
    end
    return trainingArray
end

makeTrainingData (generic function with 1 method)

In [15]:
function makeValidationData()
    numRuns = 18-15
    folder = "/home/jupyter-rubin/dosimeter/gate_data/bars/"
    trainingSize = size(filter(x->occursin("A000", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8, 8, 2, trainingSize*numRuns)
    for i = 1:trainingSize
        for r = 1:numRuns
            trainingArray[:, :, 1, (i-1)*numRuns + r] = getEdepGateBarA0(i, (r-1)*15 + 225)
            trainingArray[:, :, 2, (i-1)*numRuns + r] = getEdepGateBarA90(i, (r-1)*15 + 225)
        end
    end
    return trainingArray
end

makeValidationData (generic function with 1 method)

In [16]:
makeTrainingData()

8×8×2×7500 Array{Int64, 4}:
[:, :, 1, 1] =
 2  3   2  379  382   3  0  0
 0  2   2  304  404   2  2  0
 2  2   5  320  341   2  0  0
 2  1  23  399  375   3  4  0
 1  3   6  369  484   7  1  0
 0  2   2  374  469   4  1  2
 1  1   3  253  421   4  1  2
 1  0  29  156  193  20  3  1

[:, :, 2, 1] =
 1   3  20  462  306   2  1  1
 0   5  21  517  295   2  1  1
 0   1   2  557  406   2  1  0
 1   0   5  497  517   2  4  1
 0   0  11  421  390   4  2  2
 1   2  35  334  254   2  3  0
 1   0  27  339  417  11  0  1
 1  31   1  218  239  20  0  1

[:, :, 1, 2] =
  0  4  13  300  300   6  2  2
  1  1   4  429  344   5  1  0
  2  3   3  398  331  20  4  2
  2  2  35  536  321   2  1  1
  3  1  20  577  450   3  1  2
  1  2   5  406  504   2  1  3
  0  1   1  393  333   3  3  0
 35  2   1  126  155  11  1  0

[:, :, 2, 2] =
 2   3   5  430  380  18  1  1
 2   2   7  449  366   3  3  0
 2   1   3  472  407   8  4  1
 2   2   4  409  423   5  3  2
 1   3   4  314  350   5  1  0
 0   2  10  386  3

In [17]:
#makeValidationData()

In [18]:
function makeTargetData()
    numRuns=15
    folder = "/home/jupyter-rubin/dosimeter/gate_data/cubes/"
    trainingSize = size(filter(x->occursin("A000", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8*8*8, trainingSize*numRuns)
    for i = 1:trainingSize
        for r = 1:numRuns
            trainingArray[:,(i-1)*numRuns + r] = getEdepGateGrid(i, (r-1)*15)
        end
    end
    return trainingArray
end

makeTargetData (generic function with 1 method)

In [19]:
function makeValTargetData()
    numRuns = 18-15
    folder = "/home/jupyter-rubin/dosimeter/gate_data/cubes/"
    trainingSize = size(filter(x->occursin("A000", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8*8*8, trainingSize*numRuns)
    for i = 1:trainingSize
        for r = 1:numRuns
            trainingArray[:,(i-1)*numRuns + r] = getEdepGateGrid(i, (r-1)*15 + 225)
        end
    end
    return trainingArray
end

makeValTargetData (generic function with 1 method)

In [20]:
#makeValTargetData()

In [21]:
#makeTargetData()

In [22]:
function makeInputData()
    folder = "/home/jupyter-rubin/dosimeter/ohio_data/extracted_data/"
    trainingSize = size(filter(x->occursin("_r.txt", x), readdir(folder)))[1]
    trainingArray = Array{Int64}(undef, 8, 8, 2, trainingSize)
    for i = 1:trainingSize
        trainingArray[:,:,1,i] = getEdepOhioBarA0(i)
        trainingArray[:,:,2,i] = getEdepOhioBarA90(i)
    end
    return trainingArray
end

makeInputData (generic function with 1 method)

In [23]:
#makeInputData()

In [24]:
trainingArray = normalize(makeTrainingData())

8×8×2×7500 Array{Int64, 4}:
[:, :, 1, 1] =
 2  3   2  319  321   3  0  0
 0  2   2  256  340   2  2  0
 2  2   4  269  287   2  0  0
 2  1  19  336  316   3  3  0
 1  3   5  310  407   6  1  0
 0  2   2  315  395   3  1  2
 1  1   3  213  354   3  1  2
 1  0  24  131  162  17  3  1

[:, :, 2, 1] =
 1   3  17  389  257   2  1  1
 0   4  18  435  248   2  1  1
 0   1   2  469  342   2  1  0
 1   0   4  418  435   2  3  1
 0   0   9  354  328   3  2  2
 1   2  29  281  214   2  3  0
 1   0  23  285  351   9  0  1
 1  26   1  183  201  17  0  1

[:, :, 1, 2] =
  0  3  11  252  252   5  2  2
  1  1   3  361  289   4  1  0
  2  3   3  335  279  17  3  2
  2  2  29  451  270   2  1  1
  3  1  17  485  379   3  1  2
  1  2   4  342  424   2  1  3
  0  1   1  331  280   3  3  0
 29  2   1  106  130   9  1  0

[:, :, 2, 2] =
 2  3  4  362  320  15  1  1
 2  2  6  378  308   3  3  0
 2  1  3  397  342   7  3  1
 2  2  3  344  356   4  3  2
 1  3  3  264  294   4  1  0
 0  2  8  325  315   6  3  1

In [25]:
targetArray = makeTargetData()

512×7500 Matrix{Int64}:
 0  0  0  1  0  0  0  0  0  0  0  0  0  …   0   0  26  0  0   0  0  0  2  0
 1  0  0  0  0  1  0  0  0  0  0  0  0     17   0   8  0  0  17  0  0  0  4
 0  0  0  0  0  0  0  0  0  0  0  0  0     10   0   0  0  0   0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0      0   0   0  0  0   0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0      1   0   0  0  0   0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  …   0   0   0  1  0   0  0  0  0  0
 0  0  1  1  0  0  0  0  0  0  0  1  0      0   0   0  1  0   0  0  0  0  0
 0  0  1  0  2  0  0  0  0  0  1  0  0      0   0   0  0  0   0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1     10   4  20  0  1  22  0  0  1  0
 0  1  0  0  1  0  0  0  0  0  0  0  0      8  13   0  0  1  19  2  0  2  5
 0  0  1  0  0  0  0  0  0  0  0  0  0  …  32   3   0  0  1  19  0  1  3  5
 0  0  0  0  0  1  0  0  0  0  0  0  0     17   0   0  1  0   0  0  0  2  1
 0  0  0  0  0  0  0  0  0  0  0  0  0      1   0   0  0  0  18 

In [26]:
validationArray = normalize(makeValidationData())

8×8×2×1500 Array{Int64, 4}:
[:, :, 1, 1] =
 1  2   5  391  331  27   1   2
 2  3   2  537  408  18   2   1
 0  2  11  430  461   4   1   0
 0  2   4  341  550   3   2   1
 1  1   4  487  360   6   1   0
 3  1   4  433  399   5   3   2
 3  1   3  365  363   2   5   0
 0  1  48  162  188  25  17  28

[:, :, 2, 1] =
 1  2   3  447  587   2  1  0
 1  1   4  475  578   3  3  3
 1  1   6  390  460   6  5  2
 1  4  20  340  419   7  1  1
 0  3   3  376  364   3  4  1
 0  1   7  503  401   3  1  1
 1  1   3  273  323   5  1  1
 1  1   5  192  141  12  2  1

[:, :, 1, 2] =
 1  0  4  304  321   1  1  0
 2  2  5  251  256   3  2  0
 1  2  4  356  362   4  2  2
 0  2  5  353  554  27  3  1
 1  3  4  454  515  28  0  1
 2  4  3  461  517   1  3  2
 1  1  2  263  370   0  2  0
 1  3  1  198  147   1  1  1

[:, :, 2, 2] =
 3  1   5  411  317  0  1  1
 2  1   2  389  318  4  1  2
 3  3   2  330  426  3  2  5
 1  0   8  418  533  5  6  5
 2  4   2  372  473  4  1  0
 1  3   3  342  425  5  0  1
 1  3  

In [27]:
validationTargetArray = makeValTargetData()

512×1500 Matrix{Int64}:
 0  0  0  0  0  0  1  0  0  0  0  0  0  …  0  0  0   0  3  0  0   0  0  0  20
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0   0  0  0  0   0  0  0   0
 1  1  0  0  1  0  0  0  1  0  0  0  0     0  0  0   0  0  0  0   0  0  0   0
 0  0  0  0  0  0  0  1  0  0  0  0  0     0  0  0   0  0  0  1   1  0  0   0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0   0  0  0  0   0  0  0   0
 0  1  0  0  0  0  0  0  0  0  0  1  0  …  0  0  0   0  0  0  0   0  0  0   0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0   0  0  0  0   0  0  0   0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  0  0   0  0  0  0   0  0  0   0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0   0  5  0  3   0  0  0  10
 0  1  0  0  0  0  1  0  0  0  1  0  0     0  0  0   0  0  0  0   0  0  0   0
 1  1  0  0  0  0  0  0  0  2  0  0  0  …  0  0  0   0  0  0  3   0  0  0   0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0   0  0  0  0  13  0  0   0
 0  1  0  0  0  0  0  0  0  0  0  0  0  

In [28]:
inputArray = normalize(makeInputData())

8×8×2×21 Array{Int64, 4}:
[:, :, 1, 1] =
 31  34  43  104  173  45  35  31
 32  36  39   98  237  51  34  32
 30  33  46  157  244  47  35  32
 31  36  38  101  402  59  33  32
 33  38  46  146  287  61  35  33
 32  32  48  134  203  56  36  32
 31  31  47  109  299  50  40  31
 29  31  35   70  152  58  35  31

[:, :, 2, 1] =
 16  18  36   84   64  30  17  16
 16  20  35   72   87  44  17  16
 14  17  46  133   87  39  18  16
 16  19  30   76  119  54  17  16
 17  22  46  116  105  57  18  17
 16  16  50  107   72  51  18  16
 15  15  52   82   99  39  21  15
 14  16  25   48   54  49  18  15

[:, :, 1, 2] =
 16  18  22  120   70  20  17  16
 16  18  19  103   85  19  17  16
 15  17  23  185   96  19  17  16
 16  18  19  105  133  21  16  16
 17  19  22  163  108  22  17  17
 16  16  24  147   82  21  17  16
 16  16  23  111  135  20  19  16
 15  16  18   63   74  22  17  16

[:, :, 2, 2] =
 12  13  30   73   55  24  13  12
 12  15  29   62   75  37  13  12
 11  13  38  110   75  32  

In [29]:
function plotResults(m)
    image = rand((1: size(inputArray)[4]))
    print(image)
    display(heatmap(reshape(m(validationArray)[:,image], (8,8*8)), xlabel="Length/Height", ylabel="Width", title="GATE Prediction"))
    display(heatmap(reshape(validationTargetArray[:,image], (8,8*8)), xlabel="Length/Height", ylabel="Width", title="GATE Target"))
    display(heatmap(reshape(m(inputArray)[:,image], (8,8*8)), xlabel="Length/Height", ylabel="Width", title="Ohio Data Prediction"))
end

plotResults (generic function with 1 method)

In [30]:
#plotResults(m)

In [121]:
m = Chain(Conv((2,2), 2=>16, pad=(1,1), relu),
        x -> reshape(x, :, size(x,4)),
        Dropout(0.03),

        Dense(1296, 1800),

        Dense(1800, 8*8*8))

Chain(Conv((2, 2), 2=>16, relu), #175, Dropout(0.03), Dense(1296, 1800), Dense(1800, 512))

In [122]:
loss(x, y) = mean((m(x)-y).^2)
opt = ADAM()
dataset = (repeated((trainingArray, targetArray), 100))
evalcb = () -> @show (loss(trainingArray, targetArray)) 
CurrentR2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(trainingArray)[:,1]), B=targetArray[:,1])))
print("Training R2: ", CurrentR2, '\n')
CurrentVR2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(validationArray)[:,1]), B=validationTargetArray[:,1])))
print("Validation R2: ", CurrentVR2, '\n')

Training R2: 0.012252350387457867
Validation R2: 0.006967177505198485


In [123]:
for i = 1:10
    Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 2))
    CurrentR2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(trainingArray)[:,1]), B=targetArray[:,1])))
    print("Training R2: ", CurrentR2, '\n')
    CurrentVR2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(validationArray)[:,1]), B=validationTargetArray[:,1])))
    print("Validation R2: ", CurrentVR2, '\n')
    #plotResults(m)
end

loss(trainingArray, targetArray) = 18588.48f0
loss(trainingArray, targetArray) = 6885.636f0
loss(trainingArray, targetArray) = 3786.4214f0
loss(trainingArray, targetArray) = 2563.4111f0
loss(trainingArray, targetArray) = 2076.8997f0
loss(trainingArray, targetArray) = 1810.0184f0
loss(trainingArray, targetArray) = 1550.9272f0
loss(trainingArray, targetArray) = 1433.4678f0
loss(trainingArray, targetArray) = 1309.0891f0
loss(trainingArray, targetArray) = 1230.5898f0
loss(trainingArray, targetArray) = 1203.7517f0
loss(trainingArray, targetArray) = 1158.5579f0
loss(trainingArray, targetArray) = 1129.9674f0
loss(trainingArray, targetArray) = 1117.4913f0
loss(trainingArray, targetArray) = 1100.1058f0
loss(trainingArray, targetArray) = 1086.1986f0
loss(trainingArray, targetArray) = 1079.2242f0
loss(trainingArray, targetArray) = 1069.8752f0
loss(trainingArray, targetArray) = 1061.9305f0
loss(trainingArray, targetArray) = 1056.3146f0
loss(trainingArray, targetArray) = 1050.2722f0
loss(trainingAr

In [102]:
function writeOutput(m)
    readPath = "/home/jupyter-rubin/dosimeter/gate_data/bars/"
    readArray = filter(x->occursin(string("A000"), x), readdir(readPath))
    readSize = size(readArray)[1]
        
    path = "/home/jupyter-rubin/dosimeter/neural_nets/JuliaOutputs/RubinRuns"
    runNumber = lpad(size(readdir(path))[1], 3, "0")
    path = string(path, "/Run", runNumber, "R2-", lpad(Int(round(CurrentVR2, digits=3)*1000),3,"0"))
    mkdir(path)
    mkdir(string(path, "/fullSimulationOutputs"))
    mkdir(string(path, "/fullOhioOutputs"))
    for i = 1:size(validationTargetArray)[2]
        beamSize = filter(x->occursin(string("A000"), x), readdir(readPath))[div(100-1, div(size(validationTargetArray)[2],readSize))+1][1:8]

        writedlm(string(path,"/fullSimulationOutputs/ValidationTarget",lpad(i,3,"0"), "-", beamSize, ".txt"), reshape(validationTargetArray[:,i], (8*8,8)), " ")
        writedlm(string(path,"/fullSimulationOutputs/ValidationOutput",lpad(i,3,"0"), "-", beamSize, ".txt"), Int.(round.(reshape(m(validationArray)[:,i], (8*8,8)))), " ")
    end


    readPath = "/home/jupyter-rubin/dosimeter/gate_data/bars/"
    readArray = filter(x->occursin(string("A000"), x), readdir(readPath))
    for i = 1:size(inputArray)[4]
        writedlm(string(path,"/fullOhioOutputs/OhioOutput",lpad(i,3,"0"),".txt"), Int.(round.(reshape(m(inputArray)[:,i], (8*8,8)))), " ")
    end
end

writeOutput (generic function with 1 method)

In [124]:
writeOutput(m)